In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, LSTM, Embedding, SpatialDropout1D, Bidirectional
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical


In [77]:
# Load the balanced training data and test data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Separate features and labels
X_train = train_data.drop(columns=['hate_speech', 'lemmatized_comment'])
y_train = train_data['hate_speech']

X_test = test_data.drop(columns=['hate_speech', 'lemmatized_comment'])
y_test = test_data['hate_speech']

# Convert DataFrames to numpy arrays-keras require inpute data in numpy array format
X_train = np.array(X_train)
X_test = np.array(X_test)

# Ensure the labels are in the correct format
y_train = np.array(y_train)
y_test = np.array(y_test)

# Reshape the input data to be 3D [samples, time steps, features]
# This is necessary for Conv1D and LSTM layers which expect 3D input
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))


C:\Users\Aman Shekhar Sachan\AppData\Local\Temp\ipykernel_14036\2809165132.py:2: DtypeWarning: Columns (300) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('train.csv')


## ***Deep Learning Models***

In [21]:
# Define the CNN model
def create_cnn_model(input_shape):
    model = Sequential()
    # Use kernel size of 1 and apply padding to ensure compatibility with small input sizes
    model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# CNN Model
cnn_model = create_cnn_model((X_train.shape[1], X_train.shape[2]))
cnn_model.summary()
cnn_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_5 (Conv1D)                    │ (None, 1, 64)               │          19,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 1, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 100)                 │           6,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 25,865 (101.04 KB)

 Trainable params: 25,865 (101.04 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.6858 - loss: 0.5848 - val_accuracy: 0.6961 - val_loss: 0.5731
Epoch 2/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8235 - loss: 0.3999 - val_accuracy: 0.7751 - val_loss: 0.4983
Epoch 3/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8477 - loss: 0.3658 - val_accuracy: 0.8955 - val_loss: 0.3353
Epoch 4/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8609 - loss: 0.3358 - val_accuracy: 0.8093 - val_loss: 0.4527
Epoch 5/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8765 - loss: 0.3127 - val_accuracy: 0.8355 - val_loss: 0.4166
Epoch 6/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8747 - loss: 0.3112 - val_accuracy: 0.8626 - val_loss: 0.3634
Epoch 7/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8837 - loss: 0.2939 - val_accuracy: 0.8772 - val_loss: 0.3281
Epoch 8/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8859 - loss: 0.2825 - val_accuracy: 0

In [22]:
# Define the LSTM model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(100, return_sequences=True), input_shape=input_shape))
    model.add(SpatialDropout1D(0.5))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [23]:
# LSTM Model
lstm_model = create_lstm_model((X_train.shape[1], X_train.shape[2]))
lstm_model.summary()
lstm_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

C:\Users\Aman Shekhar Sachan\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)        │ (None, 1, 200)              │         320,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d (SpatialDropout1D) │ (None, 1, 200)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 100)                 │          20,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 341,001 (1.30 MB)

 Trainable params: 341,001 (1.30 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - accuracy: 0.6808 - loss: 0.5972 - val_accuracy: 0.5055 - val_loss: 0.8792
Epoch 2/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.7637 - loss: 0.4798 - val_accuracy: 0.7858 - val_loss: 0.4943
Epoch 3/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.7862 - loss: 0.4580 - val_accuracy: 0.7760 - val_loss: 0.5086
Epoch 4/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.7923 - loss: 0.4366 - val_accuracy: 0.7046 - val_loss: 0.6067
Epoch 5/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.7977 - loss: 0.4309 - val_accuracy: 0.7306 - val_loss: 0.5750
Epoch 6/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.8011 - loss: 0.4269 - val_accuracy: 0.7046 - val_loss: 0.6006
Epoch 7/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.8128 - loss: 0.4091 - val_accuracy: 0.8119 - val_loss: 0.4669
Epoch 8/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.8173 - loss: 0.4046 - val_acc

In [24]:
# Evaluate models
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype("int32")
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

print("\n--- CNN Model Evaluation ---")
evaluate_model(cnn_model, X_test, y_test)

print("\n--- LSTM Model Evaluation ---")
evaluate_model(lstm_model, X_test, y_test)



--- CNN Model Evaluation ---
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Accuracy: 0.8560
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.90      0.90      3386
           1       0.69      0.72      0.70      1057

    accuracy                           0.86      4443
   macro avg       0.80      0.81      0.80      4443
weighted avg       0.86      0.86      0.86      4443


--- LSTM Model Evaluation ---
139/139 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step
Accuracy: 0.8375
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.87      0.89      3386
           1       0.64      0.72      0.68      1057

    accuracy                           0.84      4443
   macro avg       0.78      0.80      0.79      4443
weighted avg       0.85      0.84      0.84      4443



### from above evalutaion we can predict that CNN performing better than LSTM